# Submit Training Job Demo

In [1]:
import json
import time
import os
import requests

import fml_manager
from fml_manager import *

The DSL and Config file can be presented in JOSN format. 
Submitting a job with JSON defined in line, can use ```submit_job(self,dsl, config)```. 
Note: the parameters are dict, the JSON string have to transform to dict with ```json.loads```

In [2]:

# dsl
secure_add_example = ComponentBuilder()\
    .with_name('secure_add_example_0')\
    .with_module('SecureAddExample').build()
        

pipeline = Pipeline(
    secure_add_example
)


# Configuration
initiator = InitiatorBuilder()\
    .with_role('guest')\
    .with_party_id(9999).build()


job_parameters = JobParametersBuilder()\
    .with_work_mode(1).build()

role = RoleBuilder()\
    .with_guest(9999)\
    .with_host(9999).build()

secure_add_example_guest_config = {
        'seed': [
          123
        ]
      }


secure_add_example_host_config = {
        'seed': [
          321
        ]
      }
role_parameters = RoleParametersBuilder()\
    .with_host_module_config(modules=['secure_add_example_0'], configs=[secure_add_example_host_config])\
    .with_guest_module_config(modules=['secure_add_example_0'], configs=[secure_add_example_guest_config]).build()

secure_add_example = {
        'partition': 10,
        'data_num': 1000
    }


algorithm_parameters = AlgorithmParametersBuilder()\
    .with_module_config(modules=['secure_add_example_0'], configs=[secure_add_example]).build()

config = ConfigBuilder()\
    .with_initiator(initiator)\
    .with_job_parameters(job_parameters)\
    .with_role(role)\
    .with_role_parameters(role_parameters)\
    .with_algorithm_parameters(algorithm_parameters).build()

manager = fml_manager.FMLManager()
response = manager.submit_job(pipeline.to_dict(), config.to_dict())

In [3]:
manager.prettify(response, verbose=True)
stdout = json.loads(response.content)
jobid = stdout['jobId']

Success!
{
    'data': {
        'board_url': 'http://fateboard:8080/index.html#/dashboard?job_id=2020063007202175889351&role=guest&party_id=9999',
        'job_dsl_path': '/data/projects/fate/python/jobs/2020063007202175889351/job_dsl.json',
        'job_runtime_conf_path': '/data/projects/fate/python/jobs/2020063007202175889351/job_runtime_conf.json',
        'logs_directory': '/data/projects/fate/python/logs/2020063007202175889351',
        'model_info': {
            'model_id': 'guest-9999#host-9999#model',
            'model_version': '2020063007202175889351'
        }
    },
    'jobId': '2020063007202175889351',
    'retcode': 0,
    'retmsg': 'success'
}


Once the job submitted, we can use ```def query_job(self, query_conditions)``` to query the status. the query_condition is a dict, and can add all job's attributes for querying.

In [4]:
query_condition = {
    'job_id':jobid
}
job_status = manager.query_job(query_condition)
manager.prettify(job_status, verbose=True)

Success!
{
    'data': [
        {
            'f_create_time': 1593501621766,
            'f_current_steps': null,
            'f_current_tasks': null,
            'f_description': '',
            'f_dsl': '{\'components\': {\'secure_add_example_0\': {\'module\': \'SecureAddExample\'}}}',
            'f_elapsed': null,
            'f_end_time': null,
            'f_initiator_party_id': '9999',
            'f_is_initiator': 1,
            'f_job_id': '2020063007202175889351',
            'f_name': '',
            'f_party_id': '9999',
            'f_progress': 0,
            'f_role': 'guest',
            'f_roles': '{\'guest\': [9999], \'host\': [9999]}',
            'f_run_ip': '192.167.0.6:9380',
            'f_runtime_conf': '{\'initiator\': {\'role\': \'guest\', \'party_id\': 9999}, \'job_parameters\': {\'work_mode\': 1, \'model_id\': \'guest-9999#host-9999#model\', \'model_version\': \'2020063007202175889351\'}, \'role\': {\'guest\': [9999], \'host\': [9999]}, \'role_parameters\'

<Response [200]>

In [5]:
manager.query_job_status(query_condition)

Status: success


'success'

We can also fetch the logs of job submitted, and save it to working folder.

In [6]:
response = manager.fetch_job_log(jobid)

{'retcode': 0, 'directory': './job_2020063007202175889351_log', 'retmsg': 'download successfully, please check ./job_2020063007202175889351_log directory, file name is job_2020063007202175889351_log.tar.gz'}
